# A Trial-and-Error Serpent Tutorial

The documentation on Serpent and Pyethereum, the Python way to do smart contracts on Ethereum, is a self-acknowledged work in progess. For people like myself who have a decent programming background, but not enough to jump right into its [source code](https://github.com/ethereum/pyethereum/) on GitHub, learning Serpent and Pyethereum means patching together existing tutorials (with occasional deprecated commands) and lots of trial and error.

By design, these tutorials are not a primer. It would even be fair to call them 'anti-primers,' since we without exception take scenic routes (via trial and error) rather than direct.

Several existing tutorials contain deprecated commands that do not work as of 2016. One tell-tale sign that a tutorial is too old to be used word-for-word is if it tells you to `import pyethereum` (as of writing, it should be `import ethereum`. Nonetheless, these tutorials are a good starting point. What follows is intended to build on these, such as

* [Ethereum programming guide blog, 2014](https://blog.ethereum.org/2014/04/10/pyethereum-and-serpent-programming-guide/) Partly out-of-date
* [University of Maryland Serpent Tutorials and Example Code](http://mc2-umd.github.io/ethereumlab/) Many examples, partly out-of-date (replace `import pyethereum` with `import ethereum`)
* [Ethereum Wiki Serpent Tutorial](https://github.com/ethereum/wiki/wiki/Serpent) Up-to-date, as far as I can tell



## Smart contracts and the importance of the "self"
Declaration, initialization and modification of variables are key capabilities for any programming language. Here, we give examples of how this does (and doesn't) work in Serpent in the simple case of declaring and incrementing an integer within a Serpent contract.

In [6]:
import serpent
from ethereum import tester as t, utils as u, abi

In [24]:
serpent_vars_basic_wrong_init = '''
data x # Declare variable x. By default, x is numeric
x = 42
    
def ask_x_selfless():
    return(x)

def ask_x_self():
    return(self.x)

def incr_x_wrong():
    x = x+1
    return(x)

def incr_x():
    self.x = self.x + 1
    return(self.x)

'''

s = t.state() # Initialize private blockchain
c_wrong = s.abi_contract(serpent_vars_basic_wrong_init) # Compile contract

('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)


In [25]:
# "self"-less 
print(c_wrong.ask_x_selfless())
print(c_wrong.incr_x_wrong()) # x only changed within function
print(c_wrong.ask_x_selfless())


('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
42
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
43
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
42


Note that querying `x` without `self` works OK, but if we want to change the definition of `x` in the contract (not just locally within the function), then the 'self-less' code does not do what we want.

In [26]:
# Now with "self"
print(c_wrong.ask_x_self())
print(c_wrong.incr_x())
print(c_wrong.ask_x_self())


('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
0
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
1
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
1


Now we are closer: our function `incr_x` does indeed change the variable value for the contract, but `ask_x_self` returns 0, not the 42 we initialized it to.

What if we mix and match, using `self` when we want to modify, but 'self-less' to query?

In [20]:
print(c_wrong.ask_x_selfless())
print(c_wrong.incr_x())
print(c_wrong.ask_x_selfless())
print(c_wrong.ask_x_self())

('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
42
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
4
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
42
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
4


So it seems that using `self` enables us to change the value of `x`, but it seems we are initializing `x` the wrong way. To correct this, we use the `init()` function in Serpent.

In [37]:
serpent_vars_basic = '''
data x # Declare variable x. By default, x is numeric

def init():
    self.x = 42
    
def ask_x_selfless():
    return(x)

def ask_x_self():
    return(self.x)

def incr_x_wrong():
    x = x+1
    return(x)

def incr_x():
    self.x = self.x + 1
    return(self.x)

'''

#s = t.state() # Commented out, since we don't need to reboot out test blockchain
c = s.abi_contract(serpent_vars_basic)

('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)


In [40]:
# "self"-less 
print(c.ask_x_selfless())
print(c.incr_x_wrong()) # x only changed within function
print(c.ask_x_selfless())


('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
0
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
1
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
0


In [41]:
# Now with "self"
print(c.ask_x_self())
print(c.incr_x())
print(c.ask_x_self())


('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
42
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
43
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
43


## Strings in Serpent functions
This I do not yet understand. See my (still unanswered) [question](http://ethereum.stackexchange.com/questions/7137/serpent-function-to-compare-strings) on Ethereum StackExchange